# Amazon Bedrock boto3 Setup

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

For more details refer to the [`Bedrock` Workshop](https://github.com/aws-samples/amazon-bedrock-workshop)

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"
%pip install "unstructured[csv]" "pillow>=9.5,<10" langchain==0.0.309 python-weather pypdf==3.8.1 ipywidgets --force-reinstall --quiet
%pip install /g/faiss-1.7.4/build/faiss/python
%pip install --upgrade setuptools

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Bedrock clients
This notebook demonstrates invoking Bedrock models directly using the AWS SDK

Here are links for some packages you might not be familiar with:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [sqlalchemy](https://www.sqlalchemy.org/), SQLAlchemy is the Python SQL toolkit 
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), for interactive UI widgets in the notebook
- [NeMo-Guardrails](https://github.com/NVIDIA/NeMo-Guardrails) an open-source toolkit for easily adding programmable guardrails to LLM-based conversational systems


---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

Depending on your environment, you might need to customize the setup when creating your Bedrock service client. To help with this, we've provided a `get_bedrock_client()` utility method that supports passing in different options. You can find the implementation in [../utils/bedrock.py](../utils/bedrock.py)

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).

The `get_bedrock_client()` method accepts `runtime` (default=True) parameter to return either `bedrock` or `bedrock-runtime` client.

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### A note about `langchain`

LangChain is a framework designed to simplify the development of applications that leverage large language models (LLMs). It provides tools and abstractions to help developers build applications that can interact with LLMs, manage their outputs, and integrate them into larger workflows.

Key features of LangChain include:

- **LLM Wrappers:** Simplified interfaces to interact with various LLMs.
- **Chains:** Sequences of calls to LLMs or other utilities, allowing for complex workflows.
- **Memory:** Mechanisms to maintain state between calls to LLMs.
- **Agents:** Components that use LLMs to make decisions and take actions.
- **Data Connectors:** Integrations with various data sources and sinks.

LangChain is particularly useful for building applications like chatbots, automated content generation tools, and other AI-driven applications that require sophisticated language understanding and generation capabilities.

The Bedrock classes provided by `langchain` create a Bedrock boto3 client by default. To customize your Bedrock configuration, we recommend to explicitly create the Bedrock client using the method below, and pass it to the [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) class instantiation method using `client=boto3_bedrock`

In [ ]:
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock #, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

model_parameter = {
    # The key "temperature" is set to 0.0. In the context of NLP models,
    # temperature is a parameter that controls the randomness of predictions
    # by scaling the logits before applying the softmax function. A lower
    # temperature (close to 0) makes the model more deterministic, producing
    # less random and more focused outputs. Conversely, a higher temperature
    # results in more random and diverse outputs.
    "temperature": 0.0,
    # The key "top_p" is set to 0.5. This parameter is used in nucleus
    # sampling, a technique for generating text. The top-p value determines
    # the cumulative probability threshold for selecting the next token. When
    # generating text, the model considers only the smallest set of tokens
    # whose cumulative probability is at least top_p. In this case, setting
    # top_p to 0.5 means the model will consider tokens that together have a
    # 50% probability of being the next token, promoting more coherent and
    # contextually relevant outputs.
    "top_p": .5,
    # The key "max_tokens_to_sample" is set to 2000. This parameter specifies
    # the maximum number of tokens the model is allowed to generate in a single
    # sampling session. Limiting the number of tokens helps control the length
    # of the generated text, ensuring it does not exceed a certain size, which
    # can be useful for managing computational resources and maintaining
    # relevance in the generated content.
    "max_tokens_to_sample": 2000
}

## Invoke Model

we will demonstrate the use of invoking the models and also highlight the default behaviour of LLM. They will return answer and data based on the training data. For our use case of AiStylist we need more curated responses. That means we will need to adopt a different architecture to get the model to execute our use case. Those techniques will be demonstrated in the next workbook

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
# customer_input = (
#     "I am a male consultant in my 30s traveling to New York next week. "
#     "What kind of outfit should I wear on my first day in the office? "
# )

customer_input = (
    "I work in my basement in gym clothes. I need to travel to a client "
    "location in a nearby city. What kind of outfit should I wear on my first "
    "day in the office? "
)

In [ ]:
# first fetch possible styles and give options to customer
# the phrase "list different styles" tells the model to output a list
# which will be parsed, and important part of using Amazon Bedrock and
# LLMs is prompt engineering
# https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
prompt = """

Human: list different style options for:
""" + customer_input + """

Assistant:"""
body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500})

# change this to use a different version from the model provider
# show the available models in the AWS console
model_id = "anthropic.claude-v2"
accept = "application/json"
content_type = "application/json"

response = bedrock_runtime.invoke_model(
    body=body,
    modelId=model_id,
    accept=accept,
    contentType=content_type
)
response_body = json.loads(response.get("body").read())
styles_response = response_body.get("completion")
print(styles_response)


## Generate the default images using stability.stable-diffusion-xl

Preparing received response to generate images for each style look

In [ ]:
# Prepare input for fetching images for each of style
styles = [
    s.strip() for s in (list(filter(None, styles_response.splitlines()))[1:-1])
]
print(*styles, sep='\n')

In [ ]:
from IPython import display
from base64 import b64decode
import json

# Fetching images for each of style

images = []
for i, style in enumerate(styles):
    if len(style.split("-")) > 1:

        # https://platform.stability.ai/docs/api-reference#tag/SDXL-1.0-and-SD1.6/operation/textToImage
        # text_prompts: A list containing a dictionary with:
        # text: A string created by concatenating "person wearing " with the
        #       second part of the style variable (split by "-").
        # weight: A float value (1.0) indicating the importance or influence
        #         of this text prompt.
        # cfg_scale: A parameter (set to 5) that likely controls the strength
        #            of adherence to the text prompt.
        # seed: A fixed integer (5450) used to initialize the random number
        #       generator for reproducibility.
        # steps: An integer (70) specifying the number of steps for the
        #        generation process, which might affect the quality or detail
        #        of the output.
        # style_preset: A string ("photographic") indicating the style or
        #               preset to be applied to the generated output.
        request = json.dumps({
            "text_prompts": (
                [{
                    "text": "person wearing " + style.split("-")[1],
                    "weight": 1.0
                }]
            ),
            "cfg_scale": 5,
            "seed": 5450,
            "steps": 70,
            "style_preset": "photographic",
        })
        model_id = "stability.stable-diffusion-xl-v1"

        response = bedrock_runtime.invoke_model(body=request, modelId=model_id)
        response_body = json.loads(response.get("body").read())

        base_64_img_str = response_body["artifacts"][0].get("base64")
        display.display(display.Image(b64decode(base_64_img_str), width=200))